In [1]:
import race_results_scanner
import google_cloud_downloader as gcd
from PyPDF2 import PdfReader
import numpy as np
import pandas as pd
import os
import header_scanner as hs


pdf = "Error_PDFS/ARP_09_18_2022.pdf"


In [2]:
#Testing for page

page_list = race_results_scanner.get_page_list(pdf)
target_page = 7

page_list_index = 0
for page in page_list:
    if(target_page == page['page_num']):
        break
    page_list_index += 1
    
reader = PdfReader(pdf) #File to be scanned
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()

page = page_list[page_list_index]
header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
for field,value in header.items():
    print(field ,':', value)
result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
print(result_tables[0])
print(result_tables[1])

The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_18_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


allowance
allowance claiming
claiming
derby trail
futurity trail
maiden
maiden claiming
maiden special weight
speed index race
stakes
starter allowance
starter optional claiming
['ARAPAHOE', 'PA', '*RK', '-', 'September', '18,', '2022', '-', 'Race', '8', 'DERBY', 'TRIAL', 'Mile', 'High', 'Derby', 'Trial', '-', 'Quarter', 'Horse', 'THREE', 'YEAR', 'OLDS', 'REMAINING', 'ELIGIBLE', 'TO', 'THE', 'TRIALS.', 'Weight', '124lbs.', 'Four', 'Hundred', 'Yards', 'On', 'The', 'Dirt', 'Current', 'Track', 'Record', '(Kuhl', 'Wave', '-', '19.149', '-', 'July', '6,', '2013)', 'Purse', '$6,900', 'Available', 'Money', '$6,900', 'Value', 'of', 'Race', '$6,900', '1st', '$4,140,', '2nd', '$1,380,', '3rd', '$690,', '4th', '$414,', '5th', '$138,', '6th', '$69,', '7th', '$69', 'Wind', 'Speed', '10', 'Wind', 'Direction', 'Head', 'Weather', 'Clear', 'Track', 'Fast', 'Off', 'at', '3', '56', 'Start', 'Good', 'for', 'all', 'Timer', 'Electronic']

track :  ARAPAHOE PA *RK
date :  September 18, 2022
race_num :  8
cri

In [4]:
#Testing for whole pdf
master_df = pd.DataFrame()
page_list = race_results_scanner.get_page_list(pdf)
print(page_list)
    
reader = PdfReader(pdf) #File to be scanned
number_of_pages = len(reader.pages) #Number of pages
header_scanner = hs.HeaderScanner()
pdf_df = pd.DataFrame()
i = 0
for page in page_list:
    header = header_scanner.scan(pdf,page['page_num']) #Header scan for page
    result_tables = race_results_scanner.scan_page(pdf,page['page_num'], page['horse_count']) #Table scan
                
    #Combine into page DF
    top_table = result_tables[0]
    bottom_table = result_tables[1]
    #Dropping horse name for merge
    bottom_table = bottom_table.drop("Horse Name", axis = 1)
    merged_df = top_table.join(bottom_table.set_index("Pgm"), on = "Pgm", rsuffix = "_RLP")
    for field,value in header.items():
        merged_df[field] = value
        #Changing all cols to object type 
        merged_df = merged_df.astype(object)
        #For every page df
    #Merge into master pdf df
    if(len(pdf_df) < 1):
        pdf_df = merged_df
    else:
        pdf_df = pd.merge(pdf_df, merged_df, how = 'outer')
            
    print(pdf_df)
    #Merging into master 
    #if(len(master_df) < 1):
     #   master_df = pdf_df
    #else:
    #    master_df = pd.merge(master_df,pdf_df, how = 'outer')

The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_18_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


[{'page_num': 0, 'horse_count': 3}, {'page_num': 1, 'horse_count': 5}, {'page_num': 2, 'horse_count': 8}, {'page_num': 3, 'horse_count': 8}, {'page_num': 4, 'horse_count': 8}, {'page_num': 5, 'horse_count': 8}, {'page_num': 6, 'horse_count': 8}, {'page_num': 7, 'horse_count': 7}, {'page_num': 8, 'horse_count': 8}]
allowance
allowance claiming
claiming
derby trail
futurity trail
maiden
maiden claiming
maiden special weight
speed index race
stakes
starter allowance
starter optional claiming
['ARAPAHOE', 'P', '*ARK', '-', 'September', '18,', '2022', '-', 'Race', '1', 'MAIDEN', 'SPECIAL', 'WEIGHT', '-', 'Arabian', 'FOR', 'MAIDENS,', 'FILLIES', 'AND', 'MARES', 'THREE', 'YEARS', 'OLD', 'AND', 'UPWARD.', 'Three', 'Year', 'Olds,', '120', 'lbs.;', 'Older,', '124', 'lbs.', 'Seven', 'Furlongs', 'On', 'The', 'Dirt', 'Current', 'Track', 'Record', '(Daring', 'Pegasus', '-', '1', '21.00', '-', 'July', '4,', '2003)', 'Purse', '$9,000', 'Available', 'Money', '$9,000', 'Value', 'of', 'Race', '$9,000', '

The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_18_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 397.897
    Last Raced  Pgm         Horse Name (Jockey)   PP Start 1/4  1/2  Str Fin  \
0  12Sep22 ARP  8.0  WMA GOT IT (Martin, David)  3.0   3.0   3  1.0  1.0   1   
1  12Sep22 ARP  5.0   SWEET-WAN (Herrera, Hugo)  1.0   2.0   2  2.0  2.0   2   
2  12Sep22 ARP  7.0     CIRCE JPH (Lopez, Jose)  2.0   1.0   1  3.0  3.0   3   

    Odds  ...                date race_num  \
0   1.00  ...  September 18, 2022        1   
1  0.90*  ...  September 18, 2022        1   
2   3.20  ...  September 18, 2022        1   

                                            criteria  \
0  FOR MAIDENS, FILLIES AND MARES THREE YEARS OLD...   
1  FOR MAIDENS, FILLIES AND MARES THREE YEARS OLD...   
2  FOR MAIDENS, FILLIES AND MARES THREE YEARS OLD...   

                  track_length   purse weather track_type off_time  \
0   Seven Furlongs On The Dirt  $9,000   Clear       Fast     1:01   
1   Seven Furlongs On The Dirt  $9,000   Clear       Fast     1:01   
2   Seven Furlongs On T

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_18_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


...

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_18_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 461.04200000000003
     Last Raced  Pgm                  Horse Name (Jockey)   PP Start  1/4  \
0   12Sep22 ARP  8.0           WMA GOT IT (Martin, David)  3.0   3.0    3   
1   12Sep22 ARP  5.0            SWEET-WAN (Herrera, Hugo)  1.0   2.0    2   
2   12Sep22 ARP  7.0              CIRCE JPH (Lopez, Jose)  2.0   1.0    1   
3    6Sep22 ARP  2.0          Chuck Nazty (Wales, Travis)  2.0   2.0    1   
4    6Sep22 ARP  3.0       Mucho Gifted (Aguilar, Daniel)  3.0   1.0    3   
..          ...  ...                                  ...  ...   ...  ...   
59   9Aug22 ARP  1.0  Dna Dashing Eagle (Morales, Adrian)  1.0   5.0  NaN   
60  23Aug22 ARP  7.0   Absolution by Fire (Soto, Jhaciel)  7.0   4.0  NaN   
61  29Aug22 PRM  4.0  Fly Donna Fly (Valles Jr., Gonzalo)  4.0   6.0  NaN   
62  22Aug22 ARP  8.0    Quick Cash Wagon (Hernandez, Rey)  8.0   8.0  NaN   
63  24Jul22 ARP  6.0                True (Garcia, Ramiro)  6.0   7.0  NaN   

    1/2  Str Fin   Odds  ...

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)
The PDF <_io.BufferedReader name='Error_PDFS/ARP_09_18_2022.pdf'> contains a metadata field indicating that it should not allow text extraction. Ignoring this field and proceeding. Use the check_extractable if you want to raise an error in this case


Found table bottom bound at 444.7080000000001
Error with top df on page 7
     Last Raced  Pgm                 Horse Name (Jockey)   PP Start  1/4  1/2  \
0   12Sep22 ARP  8.0          WMA GOT IT (Martin, David)  3.0   3.0    3  1.0   
1   12Sep22 ARP  5.0           SWEET-WAN (Herrera, Hugo)  1.0   2.0    2  2.0   
2   12Sep22 ARP  7.0             CIRCE JPH (Lopez, Jose)  2.0   1.0    1  3.0   
3    6Sep22 ARP  2.0         Chuck Nazty (Wales, Travis)  2.0   2.0    1  NaN   
4    6Sep22 ARP  3.0      Mucho Gifted (Aguilar, Daniel)  3.0   1.0    3  NaN   
..          ...  ...                                 ...  ...   ...  ...  ...   
73   5Aug22 PRM  2.0  Carisma Vista (Montalvo, Genovero)  NaN   3.0  NaN  NaN   
74          1 6  NaN                                 NaN  NaN   NaN  NaN  NaN   
75  30Aug22 ARP  7.0           Ms Eireli (Soto, Jhaciel)  NaN   6.0  NaN  NaN   
76          1 4  NaN                                 NaN  NaN   NaN  NaN  NaN   
77  23Aug22 ARP  6.0       Llego El

C:\Users\jonho\AppData\Roaming\Python\Python39\site-packages\pandas\core\reshape\merge.py:916: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  key_col = Index(lvals).where(~mask_left, rvals)


In [6]:
pdf_df.to_csv("ARAP.csv")

In [18]:
import json
with open('race_types.json') as json_file:
    race_types = json.load(json_file)

In [38]:
mc = race_types["race_types"]["maiden claiming"]["word_list"]
msp = race_types["race_types"]["maiden special weight"]["word_list"]

In [48]:
df = pd.read_csv("Last.csv",index_col = 0)


last_raced_vals = []
pgm_vals = []

for i in range(len(df)):
    if(i % 2 == 0):
        last_raced_vals.append(df.loc[i,"Last Raced Pgm"])
        pgm_vals.append(-1)
    else:
        split_val = df.loc[i,"Last Raced Pgm"].split(" ")
        pgm_vals.append(split_val[len(split_val) - 1])
        new_val = split_val[0]
        for j in range(1,len(split_val)):
            new_val = new_val + " " + split_val[j]
        last_raced_vals.append(new_val)
df = df.drop("Last Raced Pgm", axis = 1)
df.insert(loc=0, column = "Last Raced", value = last_raced_vals)
df.insert(loc=1, column = "Pgm", value = pgm_vals)
df["Pgm"] = df["Pgm"].replace(-1,np.NaN)
    

In [49]:
df

,Last Raced,Pgm,Horse Name (Jockey),Wgt M/E,PP,Start,Str 1,Str,Fin,Odds,Ind. Time,Sp. In.,Unnamed: 0,Comments
0,1 4,NaN,NaN,NaN,NaN,NaN,1/2,1,1 1/4,NaN,NaN,NaN,NaN,NaN
1,4Sep22 ARP 3,3,"Our Secret Habit (Flores, James)",124 BL b,3.0,1.0,1,1,1,4.80,20.337,88.0,NaN,broke best dre
2,6 3,NaN,NaN,NaN,NaN,NaN,1/2,1/2,1,NaN,NaN,NaN,NaN,NaN
3,24Jul22 ARP 7,7,"Elunicodecandy (Herrera, Hugo)",124 BL bf,7.0,3.0,3,2,2,1.90,20.550,83.0,NaN,lugged in throu
4,6 10,NaN,NaN,NaN,NaN,NaN,Head,1,1 3/4,NaN,NaN,NaN,NaN,NaN
5,28Aug22 ARP 5,5,"Kiss for Money (Soto, Jhaciel)",124 BL b,5.0,2.0,2,3,3,11.70,20.719,79.0,NaN,away well outfi
6,2 8,NaN,NaN,NaN,NaN,NaN,1/2,Neck,Neck,NaN,NaN,NaN,NaN,NaN
7,25Jul22 ARP 6,6,"A Mini Secret (Fonseca-Soto, Fernando)",124 BL f,6.0,5.0,4,4,4,25.00,20.994,72.0,NaN,broke well fade
8,6 8,NaN,NaN,NaN,NaN,NaN,1/2,1/2,1,NaN,NaN,NaN,NaN,NaN
9,28Aug22 ARP 2,2,"Coronas for Ivory (Lazalde, Martin)",124 BL b,2.0,4.0,5,5,5,0.90*,21.037,71.0,NaN,broke in no rall
